# SQL Sat MN  Lab#2 - Code notebooks 2: DataWrangler

In [ ]:
# Run a SQL query to populate a dataframe from our PPP table (to keep the size down)
df = spark.sql("SELECT * FROM DuckLake.ppp_loan_details WHERE BorrowerState IN ('MN','ND','SD','IA','WI') LIMIT 600000")
display(df)

In [ ]:
df.count()

In [ ]:
df_pandas = df.select("*").toPandas()
df_pandas.head(50)

## My wrangled data is below    

In [ ]:
# Code generated by Data Wrangler for pandas DataFrame

def clean_data(df_pandas):
    # Change column type to float64 for column: 'ApprovedAmount'
    df_pandas = df_pandas.astype({'ApprovedAmount': 'float64'})
    # Performed 1 aggregation grouped on columns: 'BorrowerState', 'BusinessTypeDesc'
    df_pandas = df_pandas.groupby(['BorrowerState', 'BusinessTypeDesc']).agg(ApprovedAmount_sum=('ApprovedAmount', 'sum')).reset_index()
    # Sort by columns: 'BorrowerState' (ascending), 'ApprovedAmount_sum' (descending)
    df_pandas = df_pandas.sort_values(['BorrowerState', 'ApprovedAmount_sum'], ascending=[True, False])
    return df_pandas

df_pandas_clean = clean_data(df_pandas.copy())
df_pandas_clean.head(100)

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

pd.options.display.float_format = '{:.2f}'.format

df_pandas_clean.head(100)

#### BONUS: Write your wrangled data back to a file and table

In [ ]:
# Pandas wrangled df to parquet file
df_pandas_clean.to_parquet("abfss://SQLSatFabricPrecon2024@onelake.dfs.fabric.microsoft.com/DuckLake.Lakehouse/Files/PPP/WrangledData.parquet")

In [ ]:
# Get the path by going to files - right click on the parquet file, Copy relative path
df_file = spark.read.parquet("Files/PPP/WrangledData.parquet")
df_file.write.mode("overwrite").format("delta").saveAsTable("ppp_wrangled")